In [ ]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [ ]:
from pathlib import Path
import trimesh
import pyrender
import h5py
import numpy as np
import torch
import os, os.path as osp
from tqdm import tqdm
import matplotlib.pyplot as plt

from datasets.scannet.utils_3d import ProjectionHelper, adjust_intrinsic, make_intrinsic, load_intrinsic, load_pose
from datasets.scannet.utils_3d import load_depth, load_color

In [ ]:
def get_scan_name(scene_id, scan_id):
    return f'scene{str(scene_id).zfill(4)}_{str(scan_id).zfill(2)}'

In [ ]:
subvol_size = (32, 32, 32)
voxel_size = 0.05

In [ ]:
data_dir = Path('/mnt/data/scannet/backproj')
fname = 'train50-v2.h5'
f = h5py.File(data_dir / fname, 'r')

In [ ]:
print(f.keys())
print(f['x'].shape)

In [ ]:
ndx =  50
w2g, sceneid, scanid, frames = f['world_to_grid'][ndx], f['scene_id'][ndx], f['scan_id'][ndx], f['frames'][ndx]
print(w2g)
print(sceneid, scanid)
print(frames)
g2w = torch.inverse(torch.Tensor(w2g)).numpy()

## Draw only the scene and camera

In [ ]:
scene = trimesh.scene.scene.Scene() 

# basics
scan_name = get_scan_name(sceneid, scanid)
print(scan_name)
root = Path('/mnt/data/scannet/scans')

# camera and frustum
intrinsic = make_intrinsic(1170.187988, 1170.187988, 647.75, 483.75)
intrinsic = adjust_intrinsic(intrinsic, [1296, 968], (40, 30))

pose_path = root / scan_name / 'pose' / f'{frames[0]}.txt'
pose = load_pose(pose_path).numpy()
focal = (intrinsic[0, 0], intrinsic[1, 1])
cam = trimesh.scene.Camera(name=f'{scan_name}_{frames[0]}', resolution=(40, 30), focal=focal, z_near=0.4, z_far=4.0) 

camball, campath = trimesh.creation.camera_marker(cam, marker_height=4, origin_size=0.05)
camball.apply_transform(pose)
campath.apply_transform(pose)
campath.colors = np.ones((5, 3)) * np.array((0, 255, 0))
print('Add cam')
scene.add_geometry(camball)
scene.add_geometry(campath)

# the scene mesh
scan_path = root / f'{scan_name}/{scan_name}_vh_clean_2.ply'
print(scan_path)
scan = trimesh.load(scan_path)
print(scan)
print('Add scan')
scene.add_geometry(scan)

# axes
axes = trimesh.creation.axis(axis_radius=0.1, axis_length=10)
print('Add axes')
scene.add_geometry(axes)

scene.show()

## Draw one chunk and its corresponding camera pose

In [ ]:

scene = trimesh.scene.scene.Scene() 

# basics
scan_name = get_scan_name(sceneid, scanid)
print(scan_name)
root = Path('/mnt/data/scannet/scans')

# camera and frustum
intrinsic = make_intrinsic(1170.187988, 1170.187988, 647.75, 483.75)
intrinsic = adjust_intrinsic(intrinsic, [1296, 968], (40, 30))

pose_path = root / scan_name / 'pose' / f'{frames[0]}.txt'
pose = load_pose(pose_path).numpy()
focal = (intrinsic[0, 0], intrinsic[1, 1])
cam = trimesh.scene.Camera(name=f'{scan_name}_{frames[0]}', resolution=(40, 30), focal=focal, z_near=0.4, z_far=4.0) 

camball, campath = trimesh.creation.camera_marker(cam, marker_height=4, origin_size=0.05)
camball.apply_transform(pose)
campath.apply_transform(pose)
campath.colors = np.ones((5, 3)) * np.array((0, 255, 0))
print('Add cam')
scene.add_geometry(camball)
scene.add_geometry(campath)

# the scene mesh
scan_path = root / f'{scan_name}/{scan_name}_voxelized.ply'
print(scan_path)
scan = trimesh.load(scan_path)
scan.vertices *= voxel_size
print(scan)
print('Add scan')
scene.add_geometry(scan)

# draw the chunk
# required transform is wrt the center of the box, which is what we have
# hence use the existing g2w
box = trimesh.creation.box(subvol_size, g2w)
print('Add box')
scene.add_geometry(box)

# axes
axes = trimesh.creation.axis(axis_radius=0.1, axis_length=10)
print('Add axes')
scene.add_geometry(axes)

scene.show()

## Vis only occupied voxels 

In [ ]:
scene = trimesh.scene.scene.Scene() 

scan_name = get_scan_name(sceneid, scanid)
print(scan_name)
root = Path('/mnt/data/scannet/scans')
scan_path = root / f'{scan_name}/{scan_name}_voxelized.ply'
print(scan_path)
scan = trimesh.load(scan_path)
scan.vertices *= voxel_size
print(scan)
print('Add scan')
scene.add_geometry(scan)

# get the actual subvol
x = f['x'][ndx]
voxel_dims = (1, 1, 1)

print('Draw occupied voxels')
for i in tqdm(range(subvol_size[0])):
    for j in range(subvol_size[1]):
        for k in range(subvol_size[2]):
            if x[i, j, k] == 1:
                # get the transformation of this voxel
                # w2g is wrt center of the chunk, but ijk is wrt a corner of the chunk
                # hence subtract half chunk size from ijk to get "grid coord"
                t = torch.eye(4)
                t[:3, -1] = -(torch.Tensor((i, j, k)) - 16)
                
                voxel_w2g = t @ w2g
                voxel_g2w = torch.inverse(torch.Tensor(voxel_w2g)).numpy()
                box = trimesh.creation.box(voxel_dims, voxel_g2w)
                # make the box blue
                box.visual.face_colors = np.zeros((12, 4)) + (0, 0, 255, 128)
                box.visual.vertex_colors = np.zeros((8, 4)) + (0, 0, 255, 255)
                scene.add_geometry(box)

intrinsic = make_intrinsic(1170.187988, 1170.187988, 647.75, 483.75)
intrinsic = adjust_intrinsic(intrinsic, [1296, 968], (40, 30))

pose_path = root / scan_name / 'pose' / f'{frames[0]}.txt'
pose = load_pose(pose_path).numpy()
focal = (intrinsic[0, 0], intrinsic[1, 1])
cam = trimesh.scene.Camera(name=f'{scan_name}_{frames[0]}', resolution=(40, 30), focal=focal, z_near=0.4, z_far=4.0) 
camball, campath = trimesh.creation.camera_marker(cam, marker_height=4, origin_size=0.05)
camball.apply_transform(pose)
campath.apply_transform(pose)
campath.colors = np.ones((5, 3)) * np.array((0, 255, 0))
print('Add cam')
scene.add_geometry(camball)
scene.add_geometry(campath)

axes = trimesh.creation.axis(axis_radius=0.1, axis_length=10)
print('Add axes')
scene.add_geometry(axes)

scene.show()

## project image to voxels and viz correspondences

In [ ]:
proj_img_size = (40, 30)

In [ ]:
pose_path = root / scan_name / 'pose' / f'{frames[0]}.txt'
pose = load_pose(pose_path).numpy()

In [ ]:
depth_path = root / scan_name / 'depth' / f'{frames[0]}.png' 
# invert dims in the tensor
# N, H, W -> torch nn convention
depth_big = load_depth(depth_path, (640, 480))
print(depth_big.shape)
plt.imshow(depth_big)
plt.axis('off')
plt.colorbar()

In [ ]:
depth_big.min(), depth_big.max()

In [ ]:
depth_path = root / scan_name / 'depth' / f'{frames[0]}.png' 
# invert dims in the tensor
# N, H, W -> torch nn convention
depth = load_depth(depth_path, proj_img_size)
print(depth.shape)
plt.axis('off')
plt.imshow(depth)
plt.colorbar()

In [ ]:
depth.min(), depth.max()

In [ ]:
# original RGB image
rgb_path = root / scan_name / 'color' / f'{frames[0]}.jpg' 
rgb = load_color(rgb_path, (320, 240))
print(rgb.shape)
plt.axis('off')
plt.imshow(rgb.transpose(1, 2, 0))

In [ ]:
# small RGB image
rgb_path = root / scan_name / 'color' / f'{frames[0]}.jpg' 
rgb = load_color(rgb_path, proj_img_size)
print(rgb.shape)
plt.axis('off')
plt.imshow(rgb.transpose(1, 2, 0))

## Draw correspondences between image/depth and chunk voxels

In [ ]:
# get projection
intrinsic_path = root / scan_name / 'intrinsic/intrinsic_color.txt'
intrinsic = load_intrinsic(intrinsic_path)
# adjust for smaller image size
intrinsic = adjust_intrinsic(intrinsic, [1296, 968], proj_img_size)

projection = ProjectionHelper(
            intrinsic, 
            0.4, 4.0,
            proj_img_size,
            subvol_size, voxel_size
        )

# projection expects origin of chunk in a corner
# but w2g is wrt center of the chunk -> add 16 to its "grid coords" to get the required grid coords
# ie 0,0,0 becomes 16,16,16
# add an additional translation to existing one 
t = torch.eye(4)
t[:3, -1] = 16
w2g_tmp = t @ w2g

proj3d, proj2d = projection.compute_projection(torch.Tensor(depth), torch.Tensor(pose), torch.Tensor(w2g_tmp))
print(proj3d.shape, proj2d.shape)
num_inds = proj3d[0]
print('Num correspondences:', proj3d[0], proj2d[0])
ind3d = proj3d[1:1+num_inds]
ind2d = proj2d[1:1+num_inds]
print(ind3d.min(), ind3d.max(), torch.unique(ind3d).shape, torch.prod(torch.Tensor(subvol_size)))
print(ind2d.min(), ind2d.max(), torch.unique(ind2d).shape, torch.prod(torch.Tensor(proj_img_size)))

In [ ]:
t = torch.eye(4)
t[:3, -1] = torch.Tensor((16, 16, 16))
t

In [ ]:
# number of occupied voxels
x = f['x'][ndx]
print('Occupied voxels', (x == 1).sum())

In [ ]:
# get the ijk coordinates into the chunk (array indices)
coords_3d = torch.empty(4, num_inds)
coords_3d = ProjectionHelper.lin_ind_to_coords_static(ind3d, coords_3d, subvol_size).T[:, :-1].long()
print('3d coords:', coords_3d.shape, coords_3d.dtype)
# viz in red
colors = torch.zeros(num_inds, 3, dtype=int)
colors[:, 0] = (torch.arange(num_inds) * 255 / num_inds).floor()
print('colors:', colors.shape)

In [ ]:
coords_3d.min(axis=0)[0], coords_3d.max(axis=0)[0]

In [ ]:
coords_2d = torch.empty(2, num_inds, dtype=torch.long)
coords_2d = ProjectionHelper.lin_ind2d_to_coords2d_static(ind2d, coords_2d, proj_img_size).T.numpy()

color_2d = np.zeros(proj_img_size[::-1] + (3,), dtype=int)
color_2d[coords_2d[:, 1], coords_2d[:, 0]] = colors
plt.imshow(color_2d)
plt.axis('off')

## draw occupied and mapped voxels with colors

In [ ]:
scene = trimesh.scene.scene.Scene() 

scan_name = get_scan_name(sceneid, scanid)
print(scan_name)
root = Path('/mnt/data/scannet/scans')
scan_path = root / f'{scan_name}/{scan_name}_vh_clean_2.ply'
print(scan_path)
scan = trimesh.load(scan_path)
print(scan)
print('Add scan')
scene.add_geometry(scan)

# get the actual subvol
x = f['x'][ndx]
dims = x.shape
voxel_dims = (1, 1, 1)

print('Draw occupied voxels')
for i in tqdm(range(dims[0])):
    for j in range(dims[1]):
        for k in range(dims[2]):
            if x[i, j, k] == 1:
                # get the transformation of this voxel
                # w2g is wrt center of the chunk, but ijk is wrt a corner of the chunk
                # hence subtract half chunk size from ijk to get "grid coord"
                t = torch.eye(4)
                t[:3, -1] = -(torch.Tensor((i, j, k)) - 16)
                # add an additional translation to existing one
                voxel_w2g = t @ w2g
                voxel_g2w = torch.inverse(torch.Tensor(voxel_w2g)).numpy()
                box = trimesh.creation.box(voxel_dims, voxel_g2w)
                # make the box blue
                box.visual.face_colors = np.zeros((12, 4)) + (0, 0, 255, 128)
                box.visual.vertex_colors = np.zeros((8, 4)) + (0, 0, 255, 255)
                scene.add_geometry(box)

print('Draw mapped voxels')
for coord_3d, color in tqdm(zip(coords_3d, colors)): 
    i, j, k = coord_3d.tolist()
    # get the transformation of this voxel
    # w2g is wrt center of the chunk, but ijk is wrt a corner of the chunk
    # hence subtract half chunk size from ijk to get "grid coord"
    t = torch.eye(4)
    t[:3, -1] = -(torch.Tensor((i, j, k)) - 16)
    # add an additional translation to existing one
    voxel_w2g = t @ w2g
    voxel_g2w = torch.inverse(torch.Tensor(voxel_w2g)).numpy()
    box = trimesh.creation.box(voxel_dims, voxel_g2w)

    color_tup = tuple(color.tolist())
    box.visual.face_colors = np.zeros((12, 4)) + (color_tup + (128,))
    box.visual.vertex_colors = np.zeros((8, 4)) + (color_tup + (255,))
    scene.add_geometry(box)

intrinsic = make_intrinsic(1170.187988, 1170.187988, 647.75, 483.75)
intrinsic = adjust_intrinsic(intrinsic, [1296, 968], (40, 30))

pose_path = root / scan_name / 'pose' / f'{frames[0]}.txt'
pose = load_pose(pose_path).numpy()
focal = (intrinsic[0, 0], intrinsic[1, 1])
cam = trimesh.scene.Camera(name=f'{scan_name}_{frames[0]}', resolution=(40, 30), focal=focal, z_near=0.4, z_far=4.0) 
camball, campath = trimesh.creation.camera_marker(cam, marker_height=4, origin_size=0.05)
camball.apply_transform(pose)
campath.apply_transform(pose)
campath.colors = np.ones((5, 3)) * np.array((0, 255, 0))
print('Add cam')
scene.add_geometry(camball)
scene.add_geometry(campath)

axes = trimesh.creation.axis(axis_radius=0.1, axis_length=10)
print('Add axes')
scene.add_geometry(axes)

scene.show()

## Draw all the cameras in a scene

In [ ]:
# draw all cameras in a scene
scene = trimesh.scene.scene.Scene() 

scan_name = get_scan_name(sceneid, scanid)
print(scan_name)
root = Path('/mnt/data/scannet/scans')
scan_path = root / f'{scan_name}/{scan_name}_vh_clean_2.ply'
scan = trimesh.load(scan_path)
print(scan)
print('Add scan')
scene.add_geometry(scan)

box = trimesh.creation.box(subvol_size, g2w)
print('Add box')
scene.add_geometry(box)

intrinsic = make_intrinsic(1170.187988, 1170.187988, 647.75, 483.75)
intrinsic = adjust_intrinsic(intrinsic, [1296, 968], (40, 30))
pose_files = sorted(os.listdir(root / scan_name / 'pose'), key=lambda f: int(osp.splitext(f)[0]))
pose_indices = range(0, len(pose_files), frame_skip)
print('Add poses:', len(pose_indices))
frame_skip = 40
focal = (intrinsic[0, 0], intrinsic[1, 1]) 
for ndx, pose_ndx in enumerate(tqdm(pose_indices)):
    pose_path = root / scan_name / 'pose' / pose_files[pose_ndx]
    pose = load_pose(pose_path).numpy()
    
    cam = trimesh.scene.Camera(name=f'{scan_name}_{frames[0]}', resolution=(40, 30), focal=focal, z_near=0.4, z_far=4.0) 
    camball, campath = trimesh.creation.camera_marker(cam, marker_height=4, origin_size=0.05)
    
    camball.apply_transform(pose)
    campath.apply_transform(pose)
    campath.colors = np.zeros((5, 3))
    scene.add_geometry(camball)
    scene.add_geometry(campath)

# draw the best pose in a different color
pose_path = root / scan_name / 'pose' / f'{frames[0]}.txt'
pose = load_pose(pose_path).numpy()
focal = (intrinsic[0, 0], intrinsic[1, 1])
cam = trimesh.scene.Camera(name=f'{scan_name}_{frames[0]}', resolution=(40, 30), focal=focal, z_near=0.4, z_far=4.0) 
camball, campath = trimesh.creation.camera_marker(cam, marker_height=4, origin_size=0.05)
camball.apply_transform(pose)
campath.apply_transform(pose)
# green
campath.colors = np.ones((5, 3)) * np.array((0, 255, 0))
print('Add cam')
scene.add_geometry(camball)
scene.add_geometry(campath)

axes = trimesh.creation.axis(axis_radius=0.1, axis_length=10)
print('Add axes')
scene.add_geometry(axes)

scene.show()